In [ ]:
import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric

### Hyperparameter Optimization Citeseer
#### SCAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 15, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 15, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Cora
#### SCAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SCAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
